In [ ]:
import numpy as np
import pandas as pd
import nltk
import time
import openai
openai.api_key = "TYPE HERE YOUR API KEY"

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## Determination of expectations about education and certifications with NLTK

In [ ]:
# load data and make new columns
df = pd.read_csv('LI_PM_jobDesc.csv', delimiter = ',')
df['eduExpectation'] = ['' for i in range(len(df))]
df['certiExpectation'] = ''

# definition of keywords 
education_words = ['graduation', 'degree', 'bsc', 'msc', 'ba', 'ma', 'bachelor', "master's degree"]
certification_word = ['certification']

# tokenization the jobDescriptions
for index, row in df.iterrows():
    tokens = nltk.word_tokenize(row['jobDescription'].lower())

    # definition of education expectation
    for token in tokens:
        if token in education_words:
            if df.at[index, 'eduExpectation']:
                df.at[index, 'eduExpectation'] += ', '
            df.at[index, 'eduExpectation'] += token

    # definition of certification expectation
    for token in tokens:
        if token in certification_word:
            df.loc[index, 'certiExpectation'] = token
            break

# print results
df[['companyName', 'eduExpectation', 'certiExpectation']]

## Determination of expectations about education and certifications with AI

In [ ]:
#Create a new dataframe
LI_PM_jobad = pd.read_csv('LI_PM_jobDesc.csv', delimiter = ',')
df2 = pd.DataFrame()
df2['companyName'] = LI_PM_jobad['companyName']
df2['jobDescription'] = LI_PM_jobad['jobDescription']
df2['eduExpectation'] = ''

#iterate through the dataframe
for index, row in df2.iterrows():
    
    prompt_eduExpectation = row['jobDescription'] + "/n --- /n This is a product manager job description.\
         Define what educational requirements are included in the description!\
         If the applicant must have a degree, then specify in which field the degree obtained is expected\
         and at what level. Only these two data should be included in your answer: area of degree and level of degree.\
         Answer the question as truthfully as possible using the provided text.\
         If the educational requirements cannot be clearly determined based on the description,\
         use only this value: 'Unknown' and don't say anything else."
    
    response_eduExpectation = openai.ChatCompletion.create(
        model = "gpt-4",
        messages = [
            {"role": "system", "content": "You are a helpful data science assistant."},
            {"role": "user", "content": prompt_eduExpectation}],
        max_tokens=50,
        temperature=0)
    
    message_eduExpectation = response_eduExpectation['choices'][0]['message']['content'].strip()

    df2.loc[index, 'eduExpectation'] = message_eduExpectation
    
    time.sleep(0.5)
        
    if index % 10 == 0:
        print(f"Processed row {index} for company {row['companyName']} with education expectations: {message_eduExpectation}")
        
#print results
df2[['companyName', 'eduExpectation']]

In [ ]:
#Create a new dataframe
LI_PM_jobad = pd.read_csv('LI_PM_jobDesc.csv', delimiter = ',')
df3 = pd.DataFrame()
df3['companyName'] = LI_PM_jobad['companyName']
df3['jobDescription'] = LI_PM_jobad['jobDescription']
df3['certiNeeded'] = ''
df3['certiAdvantage'] = ''

#iterate through the dataframe
for index, row in df3.iterrows():
    
    prompt_certiNeeded = row['jobDescription'] + "/n --- /n This is a product manager job description.\
         Define what certifications applicants definitely need.\
         Answer the question as truthfully as possible using the provided text.\
         If the needed certifications cannot be clearly determined based on\
         the description, use only this value: 'Unknown', and don't use whole sentences."

    prompt_certiAdvantage = row['jobDescription'] + "/n --- /n This is a product manager job description.\
         Define what certifications would be an advantage for applicants.\
         Answer the question as truthfully as possible using the provided text.\
         If the advantageous certifications cannot be clearly determined based on\
         the description, use only this value: 'Unknown', and don't use whole sentences."
    
    response_certiNeeded = openai.ChatCompletion.create(
        model = "gpt-4",
        messages = [
            {"role": "system", "content": "You are a helpful data science assistant."},
            {"role": "user", "content": prompt_certiNeeded}],
        max_tokens=50,
        temperature=0)
    
    response_certiAdvantage = openai.ChatCompletion.create(
        model = "gpt-4",
        messages = [
            {"role": "system", "content": "You are a helpful data science assistant."},
            {"role": "user", "content": prompt_certiAdvantage}],
        max_tokens=50,
        temperature=0)
    
    message_certiNeeded = response_certiNeeded['choices'][0]['message']['content'].strip()
    message_certiAdvantage = response_certiAdvantage['choices'][0]['message']['content'].strip()

    df3.loc[index, 'certiNeeded'] = message_certiNeeded
    df3.loc[index, 'certiAdvantage'] = message_certiAdvantage
    
    time.sleep(0.5)
        
    if index % 10 == 0:
        print(f"Processed row {index} for company {row['companyName']} with needed certifications: {message_certiNeeded} and with advantageous certifications: {message_certiAdvantage}")
        
#print results
df3[['companyName', 'certiNeeded', 'certiAdvantage' ]]

## Further analysis with chatGPT

At first, I classified the text fragments collected by the AI from the 212 job advertisements into 5 large groups. Then, I copied these groups one by one to ChatGPT and asked it to help me with the analysis using the following prompt:

"I will show some job advertisement details about fields of education/education levels/certifications given by employers - from now on, I will refer to this group of advertisements as jobAd_n.
Tasks:
1) Combine similar fields of education/education levels/certifications in the jobAd_n advertisement group!
2) Then, collect the 15 most frequently occurring fields of education/education levels/certifications from this jobAd_n group!
3) Finally, make a table of how many advertisements appeared in the jobAd_n group!"

Then, I saved the table created by ChatGPT for myself and performed the same analysis with all advertisement groups. Finally, I copied the tables created after all 5 groups and asked ChatGPT to combine similar fields of education/education levels/certifications and make a last combined table.